Задание 1 - в общем такая проблема что если я ищу внутри статьи то есть по тексту, то у меня в dataframe почему то записываются дубликаты найденной этой страницы и никак не могу понять почему. Поэтому когда я создаю df то список заворачиваю в set чтобы избваться от дублей. Подскажите плз что не так в коде?

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

In [66]:
KEYWORDS = ['itsoft','syllabus','здоровье']
my_list = []

def get_all(pages):
    for i in range(1,pages+1):
        req = requests.get('https://habr.com/ru/all/page{}'.format(i))
        time.sleep(0.2)
        soup = BeautifulSoup(req.text,'html.parser')
        posts = soup.find_all('article',class_='post')
        for post in posts:
            post_id = post.parent.attrs.get('id')
            if not post_id:
                continue
            post_id = int(post_id.split('_')[-1])
            links = post.find('a', class_='post__title_link').attrs.get('href')
            req2 = requests.get(links)
            soup2 = BeautifulSoup(req2.text,'html.parser')
            text = soup2.find('div', id='post-content-body').text
            hubs = post.find_all('a', class_='hub-link')
            for hub in hubs:
                hub_lower = hub.text.lower()
                for keyw in KEYWORDS:
                    if keyw.lower() in hub_lower or keyw.lower() in text.lower():
                        title_element = post.find('a', class_='post__title_link')
                        date_element = post.find('span',class_='post__time')
                        key = keyw
                        page = i
                        link = title_element.attrs.get('href')
                        full = date_element.text,title_element.text,link,key,page,text
                        my_list.append(full)
                        #print(full)

In [67]:
get_all(1)

In [71]:
df = pd.DataFrame(set(my_list),columns=['date','title','link','keyword','page','text'])

In [72]:
df

,date,title,link,keyword,page,text
0,вчера в 19:18,Конструирование эпидемиологических моделей,https://habr.com/ru/post/551682/,здоровье,1,Эпидемиология из-за некоторого стечения обстоя...
1,вчера в 18:49,Визуализация данных в интерфейсе,https://habr.com/ru/company/epam_systems/blog/...,здоровье,1,"Меня зовут Илона, я Senior Experience Designer..."
2,вчера в 17:50,Агент 007: как слова-призраки защищают английс...,https://habr.com/ru/company/englishdom/blog/55...,syllabus,1,Даже в самых известных словарях бывают ошибки....
3,сегодня в 06:45,Программа для создания deb-пакетов,https://habr.com/ru/company/itsoft/blog/551556/,itsoft,1,Обычно для создания deb-пакетов используются к...


Задание 2

In [16]:
header = {
    'Accept': 'application/json, text/plain, */*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
    'Connection': 'keep-alive',
    'Content-Length': '31',
    'Content-Type': 'application/json;charset=UTF-8',
    'Host': 'identityprotection.avast.com',
    'Origin': 'https://www.avast.com',
    'Referer': 'https://www.avast.com/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36 OPR/75.0.3969.171',
    'Vaar-Header-App-Build-Version': '1.0.0',
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast',
    'Vaar-Version': '0'
}


EMAIL = ['xxx@x.ru', 'ahmediev17@gmail.com','yyy@y.com']
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

my_list = []
for item in EMAIL:
    response = requests.post(url, headers=header, json={'emailAddresses':[item]})
    time.sleep(0.2)
    for breach in response.json()['breaches'].values():
        my_list.append({
            'почта': item,
            'дата утечки':breach['publishDate'],
            'источник утечки': breach['site'],
            'описание утечки': breach['description']
        })

In [17]:
my_list

[{'почта': 'xxx@x.ru',
  'дата утечки': '2019-03-28T00:00:00Z',
  'источник утечки': 'verifications.io',
  'описание утечки': 'Big data e-mail verification platform verifications.io leaked a database containing sensitive PII belonging to over 600 million victims. The breach was discovered by a security researcher who found an unsecured 150GB MongoDB database, which he was able to track to an email verification service called Verifications.io. The leaked database contained information such as physical addresses, phone numbers, email addresses, dates of birth, gender, employer and job information, geographic location and IP addresses.\r\n\r\n'},
 {'почта': 'xxx@x.ru',
  'дата утечки': '2020-05-21T00:00:00Z',
  'источник утечки': 'vk.com',
  'описание утечки': 'At some time in 2020, the Russian social networking website vKontakte was allegedly breached. The stolen data contains email addresses and additional personal information. This breach is being privately shared on the internet.'},
 

In [18]:
df1 = pd.DataFrame(my_list,columns=['почта','дата утечки','источник утечки','описание утечки'])

In [19]:
df1

,почта,дата утечки,источник утечки,описание утечки
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,ahmediev17@gmail.com,2018-08-24T00:00:00Z,chegg.com,"In 2018, education tech company Chegg's databa..."
9,yyy@y.com,2020-01-03T00:00:00Z,azcentral.com,"At an unconfirmed date, online Arizona newspap..."
